# 1. Threading

In [10]:
import threading

t1 = threading.Thread(target=print, args=("hello thread 1",))
t2 = threading.Thread(target=print, args="hello thread 2")

# t1.start()
# t2.start()

t2.start()
t1.start()

h e l l o   t h r e a d   2
hello thread 1


In [2]:
import threading
import time

message = "hello thread"
t1 = threading.Thread(target=print, args=(message,))
t1.start()


def my_print(message):
    print(message)
    for f in (len, id, type):
        print(f(message))


t1 = threading.Thread(target=my_print, args=(message,))
t1.start()

hello thread
hello thread
12
4398073456
<class 'str'>


In [ ]:
import threading

t1 = threading.Thread(target=print, args=("hello thread 3",))
t2 = threading.Thread(target=print, args="hello thread 4")

t2.start()
t1.start()

In [ ]:
import threading


class MyThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(MyThread, self).__init__(*args, **kwargs)

    def run(self):
        print("called by threading.Thread.start()")


if __name__ == "__main__":
    mythread = MyThread()
    mythread.start()
    mythread.join()

In [13]:
import threading
import time


def my_print(*args):
    print("Enter my print")
    print(args)
    time.sleep(1)
    print("Leave my print")


if __name__ == "__main__":
    print("Enter main")

    dm_thread1 = threading.Thread(
        target=my_print, args=("hello Daemon",), daemon=None)
    dm_thread1.start()

    print("Quit main")


Enter main
Enter my print
('hello Daemon',)
Quit main


Leave my print


In [12]:
import threading
import time


def my_print(*args):
    print("Enter my print")
    print(args)
    time.sleep(1)
    print("Leave my print")


if __name__ == "__main__":
    print("Enter main")

    dm_thread1 = threading.Thread(
        target=my_print, args=("hello Daemon",), daemon=True)
    dm_thread1.start()

    print("Quit main")


Enter main
Enter my print
('hello Daemon',)
Quit main


Leave my print


In [2]:
import threading
import time


def my_print1(*args):
    print("Enter my print1")
    print(args)
    time.sleep(1)
    print("Leave my print1")


def my_print2(*args):
    print("Enter my print2")
    print(args)
    time.sleep(1)
    print("Leave my print2")


t1 = threading.Thread(target=my_print1, args=("hello thread 1",))
t2 = threading.Thread(target=my_print2, args="hello thread 2")

t1.start()
t2.start()

# t2.join()
# t1.join()

print("Quit main.")

Enter my print1
('hello thread 1',)
Enter my print2
('h', 'e', 'l', 'l', 'o', ' ', 't', 'h', 'r', 'e', 'a', 'd', ' ', '2')
Quit main.


Leave my print1
Leave my print2


In [ ]:
import threading
import time


def print_to_console(msg, lock):
    """
    test
    """
    lock.acquire()
    print(f"Enter {msg}")
    time.sleep(1)
    print(f"Leave {msg}")
    lock.release()


msg = [f"hello thread {x}" for x in range(1, 4)]
lock = threading.Lock()

t1 = threading.Thread(target=print_to_console, args=(msg[0], lock))
t2 = threading.Thread(target=print_to_console, args=(msg[1], lock))
t3 = threading.Thread(target=print_to_console, args=(msg[2], lock))

t3.start()
t2.start()
t1.start()

# 2. Multiprocessing

In [ ]:
from multiprocessing import Process
import os


def info(title):
    print(title)
    print("module name:", __name__)
    print("parent process:", os.getppid())
    print("process id:", os.getpid())


def f(name):
    info("function f")
    print("hello", name)


if __name__ == "__main__":
    info("main line")
    p = Process(target=f, args=("bob",))
    p.start()
    p.join()

In [ ]:
import multiprocessing as mp
print(mp.cpu_count())
print(mp.active_children())
print(mp.current_process())

In [ ]:
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    mp.set_start_method('spawn')
    q = mp.Queue()
    p = mp.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()


In [ ]:
import multiprocessing as mp

def foo(q):
    q.put('hello')

if __name__ == '__main__':
    ctx = mp.get_context('spawn')
    q = ctx.Queue()
    p = ctx.Process(target=foo, args=(q,))
    p.start()
    print(q.get())
    p.join()

In [ ]:
from multiprocessing import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 seconds
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")

In [ ]:
from multiprocessing import Pool
p = Pool(5)
def f(x):
    return x*x

with p:
    p.map(f, [1,2,3])

In [ ]:
import multiprocessing, time, signal
mp_context = multiprocessing.get_context('spawn')
p = mp_context.Process(target=time.sleep, args=(1000,))
print(p, p.is_alive())

p.start()
print(p, p.is_alive())

p.terminate()
time.sleep(0.1)
print(p, p.is_alive())

p.exitcode == -signal.SIGTERM

# 3. Concurrent.Futures

https://www.packetswitch.co.uk/what-is-concurrent-futures-and-how-can-it-boost-your-python-performance/

In [ ]:
import time
import random

def mail_letter(letter):
    duration = random.randint(1, 5)
    print(f"Started mailing letter {letter} (duration: {duration}s)")
    time.sleep(duration)
    print(f"Finished mailing letter {letter}")
    return f"Letter {letter} mailed"

if __name__ == '__main__':
    letters = ['A', 'B', 'C', 'D', 'E']
    results = []

    for letter in letters:
        result = mail_letter(letter)
        results.append(result)

    print("Mailing Results:")
    for result in results:
        print(result)


In [2]:
import concurrent.futures
import time
import random

def mail_letter(letter):
    duration = random.randint(1, 5)
    print(f"Started mailing letter {letter} (duration: {duration}s)")
    time.sleep(duration)
    print(f"Finished mailing letter {letter}")
    return f"Letter {letter} mailed"

if __name__ == '__main__':
    letters = ['A', 'B', 'C', 'D', 'E']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(mail_letter, letters))

    print("Mailing Results:")
    for result in results:
        print(result)


Started mailing letter A (duration: 4s)Started mailing letter B (duration: 5s)

Started mailing letter C (duration: 2s)
Started mailing letter D (duration: 2s)
Started mailing letter E (duration: 4s)
Finished mailing letter C
Finished mailing letter D
Finished mailing letter A
Finished mailing letter E
Finished mailing letter B
Mailing Results:
Letter A mailed
Letter B mailed
Letter C mailed
Letter D mailed
Letter E mailed


In [4]:
import concurrent.futures
import time
import random

def mail_letter(letter):
    duration = random.randint(1, 5)
    print(f"Started mailing letter {letter} (duration: {duration}s)")
    time.sleep(duration)
    print(f"Finished mailing letter {letter}")
    return f"Letter {letter} mailed"

if __name__ == '__main__':
    letters = ['A', 'B', 'C', 'D', 'E']

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(mail_letter, letter): letter for letter in letters}

        for future in concurrent.futures.as_completed(futures):
            letter = futures[future]
            result = future.result()
            print(f"Result: {result}")

Started mailing letter A (duration: 5s)
Started mailing letter B (duration: 2s)
Started mailing letter C (duration: 2s)
Started mailing letter D (duration: 3s)
Started mailing letter E (duration: 5s)
Finished mailing letter BFinished mailing letter C
Result: Letter C mailed

Result: Letter B mailed
Finished mailing letter D
Result: Letter D mailed
Finished mailing letter AFinished mailing letter E

Result: Letter E mailed
Result: Letter A mailed


# 4. AysncIO

It is not allowed in Jupyter notebook.

In [4]:
top = (
    'shanghai'
    'jiao'
    'tong'
)
print(top, type(top))

('shanghai', 'jiao', 'tong') <class 'tuple'>


In [ ]:
# import asyncio


# async def count():
#     print("One")
#     await asyncio.sleep(1)
#     print("Two")

# async def main():
#     await asyncio.gather(count(), count(), count())

# if __name__ == "__main__":
#     import time
#     s = time.perf_counter()
#     asyncio.run(main())
#     elapsed = time.perf_counter() - s
#     print(f"{__file__} executed in {elapsed:0.2f} seconds.")

# 5. RE

In [ ]:
import re

p = re.compile('[b-d]')
print(p, type(p))

txt = "cs1a2b3c4d5e6f7z"

m = p.match(txt)
print(m, type(m))

m = p.search(txt)
print(m, type(m))

m = p.findall(txt)
print(m, type(m))

m = p.finditer(txt)
print(m, type(m))
for _ in m:
    print(_)

In [ ]:
import re
p = re.compile('[a-z]+')

m = p.match('tempo')

print(m.group())
print(m.start(), m.end())
print(m.span())

In [ ]:
import re
p = re.compile('[a-z]+')
print(p.match('::: message'))
m = p.search('::: message')
print(m)
print(m.group())
print(m.span())

In [ ]:
import re

print(re.match(r'From\s+', 'Fromage amk'))
print(re.match(r'From\s+', 'From amk Thu May 14 19:12:10 1998'))

print(re.match('[a-z]+', '::: message'))
print(re.search('[a-z]+', '::: message'))